In [66]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain.memory.buffer_window import ConversationBufferWindowMemory
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.callbacks.manager import get_openai_callback

In [2]:
os.environ['OPENAI_API_KEY']=os.environ.get('OPENAI_API_KEY')

In [67]:
model = ChatOpenAI()

In [11]:
# loading csv file
loader = DirectoryLoader('dataset/', glob="**/*.csv")
documents = loader.load()

In [50]:
# Load CSV files
csv_loader = DirectoryLoader('dataset', glob="**/*.csv")
pdf_loader = PyPDFLoader('dataset/improving-mental-health.pdf') 

all_documents = []
all_documents.extend(csv_loader.load())
all_documents.extend(pdf_loader.load())

In [51]:
# splitting the document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                      chunk_overlap=300)
texts = text_splitter.split_documents(all_documents)

In [52]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(texts, embedding_function)

/home/employee/Desktop/Hachathon/mha-llm/hackvenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [46]:
# query it
query = "How do I handle work pressure?"
docs = db.similarity_search(query)

In [64]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
llm.predict("Hello world!")

'Hello! How can I assist you today?'

In [65]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# # Run chain
# question = "I can't sleep"
# qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(max_tokens=512),
#                                        retriever=db.as_retriever(search_type='mmr'),
#                                        return_source_documents=True,
#                                        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


# result = qa_chain({"query": question})
# result["result"]

" Thanks for asking! Poor sleep can affect your mood, outlook, and patience. Stress and anxiety can also have a negative impact on sleep. It's important to be proactive and practice healthy sleep habits. You may also want to check in with your doctor or healthcare team to see if there are any additional measures or precautions you should take, especially if you have a chronic illness. Remember to plan ahead and make sure you have any necessary medications, supplies, or supplements on hand. Don't hesitate to seek out therapy or support to help you manage any underlying stress or anxiety that may be causing your sleep issues."

In [78]:
# from langchain.memory import ConversationBufferMemory
# memory = ConversationBufferMemory(
#     memory_key="chat_history",
#     return_messages=True
# )

In [86]:
retriever=db.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm=OpenAI(),
    retriever=retriever,
    return_source_documents=True,
    return_generated_question=True,
    
)
chat_history = []

In [87]:
question = "What techniques or methods have you found most effective for managing symptoms of social anxiety?"
result = qa({"question": question, "chat_history": chat_history})
chat_history.extend([(question, result["answer"])])
print(result['answer'])

 Some techniques that have been found effective for managing symptoms of social anxiety include cognitive-behavioral therapy, exposure therapy, relaxation techniques, and mindfulness practices. It may also be helpful to talk to a therapist or counselor about your specific anxieties and develop a personalized plan for managing them.


In [88]:
print(chat_history)

[('What techniques or methods have you found most effective for managing symptoms of social anxiety?', ' Some techniques that have been found effective for managing symptoms of social anxiety include cognitive-behavioral therapy, exposure therapy, relaxation techniques, and mindfulness practices. It may also be helpful to talk to a therapist or counselor about your specific anxieties and develop a personalized plan for managing them.')]


In [89]:
question = "Can you elaborate any one of them ?"
result = qa({"question": question, "chat_history": chat_history})
chat_history.extend([(question, result["answer"])])
print(result['answer'])


 One specific technique for managing symptoms of social anxiety is exposure therapy. This involves gradually exposing oneself to feared social situations in a safe and controlled manner, with the support and guidance of a therapist. This can help to desensitize the individual to their fears and build confidence and coping skills. It is important to note that this technique should be done under the guidance of a trained therapist, as it can be difficult and potentially harmful to attempt on one's own.


In [90]:
print(chat_history)

[('What techniques or methods have you found most effective for managing symptoms of social anxiety?', ' Some techniques that have been found effective for managing symptoms of social anxiety include cognitive-behavioral therapy, exposure therapy, relaxation techniques, and mindfulness practices. It may also be helpful to talk to a therapist or counselor about your specific anxieties and develop a personalized plan for managing them.'), ('Can you elaborate any one of them ?', " One specific technique for managing symptoms of social anxiety is exposure therapy. This involves gradually exposing oneself to feared social situations in a safe and controlled manner, with the support and guidance of a therapist. This can help to desensitize the individual to their fears and build confidence and coping skills. It is important to note that this technique should be done under the guidance of a trained therapist, as it can be difficult and potentially harmful to attempt on one's own.")]


In [91]:
question = "Can you elaborate one more?"
result = qa({"question": question, "chat_history": chat_history})
chat_history.extend([(question, result["answer"])])
print(result['answer'])

 Another technique for managing symptoms of social anxiety is to practice cognitive-behavioral therapy (CBT). In CBT, individuals learn to identify and challenge negative thoughts and beliefs that contribute to their anxiety. This can involve techniques such as thought records, where individuals write down their negative thoughts and then challenge them with more realistic and positive thoughts. CBT also involves gradual exposure to feared situations, allowing individuals to face their fears in a controlled and supportive environment. This can help individuals build confidence and reduce the intensity of their anxiety in social situations.


In [92]:
print(chat_history)

[('What techniques or methods have you found most effective for managing symptoms of social anxiety?', ' Some techniques that have been found effective for managing symptoms of social anxiety include cognitive-behavioral therapy, exposure therapy, relaxation techniques, and mindfulness practices. It may also be helpful to talk to a therapist or counselor about your specific anxieties and develop a personalized plan for managing them.'), ('Can you elaborate any one of them ?', " One specific technique for managing symptoms of social anxiety is exposure therapy. This involves gradually exposing oneself to feared social situations in a safe and controlled manner, with the support and guidance of a therapist. This can help to desensitize the individual to their fears and build confidence and coping skills. It is important to note that this technique should be done under the guidance of a trained therapist, as it can be difficult and potentially harmful to attempt on one's own."), ('Can you